In [1]:
from midori64 import *
import random
from tqdm import tqdm

In [2]:
random.seed(0xdead)

In [3]:
keys = [
    [random.randint(0, 15) for _ in range(16)]
    for _ in range(10)
]

# Day 1 - Morning

![](./midori.dark.drawio.png)

## Question 1

Evaluer la propriete _balanced_ sur les 4 bits
- Après 1 tour, quels sont les nibbles equilibrés ?
- Après 2 tour, quels sont les nibbles equilibrés ?
- Après 3 tour, quels sont les nibbles equilibrés ?
- Après 4 tour, quels sont les nibbles equilibrés ?

In [4]:
def draw_balanced(arr, false_char = '░', true_char = '█', label=None):
    '''
    Draws a grid filling up the balanced cells
    '''
    for y in range(4):
        for x in range(4):
            c = true_char if arr[y * 4 + x] else false_char
            print(f'{c}', end='')
        print('\n', end='')
    print(f'{label}')

In [5]:
msgs = [[random.randint(0, 15) for _ in range(16)] for _ in range(64)]
# msg = [15, 6, 15, 2, 0, 13, 12, 15, 10, 7, 11, 2, 2, 6, 11, 1]

for r in range(1, 5):
    isBalanced = [True for _ in range(16)]

    for msg in msgs:
        cts = []

        for nibble in range(16):
            m = [x for x in msg]
            m[0] = nibble
            cts.append(encrypt(m, r, keys))


        res = [0 for _ in range(16)]
        for key_guess in range(16):
            for ct in cts:
                res[key_guess] ^= ct[key_guess]

        isBalanced = [(r & 0b1111) == 0  and b for r, b in zip(res, isBalanced)]

    draw_balanced(isBalanced, label=f'Round {r}\n')

████
████
████
████
Round 1

████
████
████
████
Round 2

████
████
████
████
Round 3

█░░░
░░░░
░░░░
░░░░
Round 4



Balanced cells are represented as `'█'` whereas unbalanced cells are represented as `'░'`

Running this cell multiple times, we realize that:
- **in rounds 1, 2 and 3 all the cells are balanced**.
- In round 4, the balanced rounds change with different executions. It seems that only the **first cell is balanced**

## Question 2

Monter une attaque en utilisant le distingueur:
- Choisir un nombre de tours R et un nible equilibré
- Ajouter un tour et retrouver les bits de clé impliqué dans le déchiffrement ?.

In order to only to only have to guess one nibble of the key at a time, we take advantage of the linearity of MixColumn and ShuffleCell to move the key earlier in the process (see diagram).

We verify for every key position and every possible value if the cells are still balanced

![](./midori_square_attack3.dark.drawio.png)

In [6]:
def is_balanced(msgs, draw=False):
    '''
    Complexity ~ 16 * msgs(len)
    '''
    res = [0 for _ in range(16)]
    for i in range(16):
        for msg in msgs:
            res[i] ^= msg[i]

    isBalanced = [(r & 0b1111) == 0 for r in res]
    if draw:
        draw_balanced(isBalanced)
    return isBalanced

In [7]:
# We wil attempt to retrieve the round key 4
# By reverting one round, we should be balanced
msgs = [[random.randint(0, 15) for _ in range(16)] for _ in range(16)]

GUESSED_KEY = [0 for _ in range(16)]


# Builds a table of delta_sets, inverting mixColumns and shuffleCells
delta_sets = []

# Iterate over messages
for msg in msgs:
    '''
    complexity ~ len(msgs) * 4 * 16
    '''
    # Change nibble position to guess bytes 6, 7, 9 and 11
    for n_pos in range(4):
        delta_set = []

        for nibble in range(16):
            m = [x for x in msg]
            m[n_pos] = nibble

            ct = encrypt(m, 4, keys)
            # Revert one turn
            delta_set.append(invShuffleCell(mixColumns(ct)))

        delta_sets.append(delta_set)


# Iterate over key positions
for key_pos in tqdm(range(16)):
    '''
    Complexity ~ 16 * 16 * (len(delta_sets) = 64) * (len(delta_set) = 16)
    '''
    k4_guess = [0 for _ in range(16)]

    # Iterate over key guesses
    for key_guess in range(16):
        k4_guess[key_pos] = key_guess

        for delta_set in delta_sets:
            guesses = [subCell(addRoundKey(guess, k4_guess)) for guess in delta_set]

            if not is_balanced(guesses)[key_pos]:
                break

        else:
            # If it is balanced for all positions and messages it's probably right
            GUESSED_KEY[key_pos] = key_guess
            break

100%|██████████| 16/16 [00:00<00:00, 217.02it/s]


In [8]:
assert keys[3] == mixColumns(shuffleCell(GUESSED_KEY))

This attack succeeds with a complexity of ~$2^{14}$ instead of ~$2^{64}$ (complexity of brute force on 4 turns)

## Question 3

Ajouter un tour avant et faire les sommes partielles

We want to add a new turn at the very beginning which will then provide us with the delta set we had in the previous exercise. The following graph helps to understand the attack
![](./midori_square_attack4_dark.drawio.png)

In [9]:
# c to speed up large encrypt requests
import midori64_c

# We wil attempt to retrieve the round key 5
# Using partial sums to reduce the amount of cipher texts
msgs = [[random.randint(0, 15) for _ in range(16)] for _ in range(8)]

GUESSED_KEY = [0 for _ in range(16)]

# Builds a containing the partial sums for each message
delta_sets = []

# Iterate over messages
for msg in tqdm(msgs):
    '''
    Complexity ~ 2 * 16 * 16 * 16 * (len(msgs) = 8) * 16
    '''
    # iterate over nibble positions like in the previous exercise
    for n_pos in range(2): # we don't need all the indexes

        # Create our partial sums
        delta_set = [[0 for _ in range(16)] for _ in range(16)]

        for nibble1 in range(16):
            for nibble2 in range(16):
                for nibble3 in range(16):
                    m = [x for x in msg]
                    match n_pos:
                        case 0:
                            m[7] = nibble1
                            m[14] = nibble2
                            m[9] = nibble3
                        case 1:
                            m[10] = nibble1
                            m[3] = nibble2
                            m[13] = nibble3
                        case 2:
                            m[5] = nibble1
                            m[11] = nibble2
                            m[2] = nibble3
                        case 3:
                            m[15] = nibble1
                            m[1] = nibble2
                            m[6] = nibble3

                    ct = midori64_c.encrypt(m, 5, keys)

                    # Revert one turn
                    ct = invShuffleCell(mixColumns(ct))

                    for i in range(16):
                        delta_set[i][ct[i]] ^= 1

        delta_sets.append(delta_set)


# Iterate over key positions
for key_pos in range(16):
    '''
    Complexity ~16 * (len(delta_sets) = 16) * 16
    '''
    print(f'Guessing key index {key_pos} (correct value: {invShuffleCell(mixColumns(keys[4]))[key_pos]})')
    # Iterate over key guesses
    for key_guess in range(16):
        for delta_set in delta_sets:
            res = 0
            for i in range(16):
                res ^= delta_set[key_pos][i] * S_BOX[i ^ key_guess]

            if res != 0:
                break

        else:
            # If it is balanced for all positions and messages it's probably right
            GUESSED_KEY[key_pos] = key_guess
            print(key_guess, end=' ')
    print('\n')

 12%|█▎        | 1/8 [00:00<00:04,  1.59it/s]

100%|██████████| 8/8 [00:04<00:00,  1.77it/s]

Guessing key index 0 (correct value: 7)
5 7 13 15 

Guessing key index 1 (correct value: 9)
1 3 9 11 

Guessing key index 2 (correct value: 4)
4 6 12 14 

Guessing key index 3 (correct value: 8)
0 2 8 10 

Guessing key index 4 (correct value: 7)
5 7 13 15 

Guessing key index 5 (correct value: 0)
0 2 8 10 

Guessing key index 6 (correct value: 13)
5 7 13 15 

Guessing key index 7 (correct value: 13)
5 7 13 15 

Guessing key index 8 (correct value: 4)
4 6 12 14 

Guessing key index 9 (correct value: 4)
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 

Guessing key index 10 (correct value: 10)
0 2 8 10 

Guessing key index 11 (correct value: 12)
4 6 12 14 

Guessing key index 12 (correct value: 4)
4 6 12 14 

Guessing key index 13 (correct value: 1)
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 

Guessing key index 14 (correct value: 8)
0 2 8 10 

Guessing key index 15 (correct value: 12)
4 6 12 14 



Unfortunately, we do not have enough constraints to conclude... I feel like we might be missing key relations at this point..

In my experimentation I found that different strategies are able to find the key, but none of them corresponded exactly to the expected strategy.

The complexity of this attack is ~$2^{20}$ instead of the ~$2^{80}$ (complexity of brute force on 5 turns), it's only a power of $2^6\times$ more expensive than our previous attack whereas brute force would of cost a factor $2^{16}$.

## Observation

I initially inverted the side of the multiplication for `mixColumns` on my paper so used the indexes `0, 10, 5, 15`. Doing so actually provides the 5th round key. I am unsure why though. This attack has complexity ~$2^{23}$

In [10]:
# c to speed up large encrypt requests
import midori64_c

# We wil attempt to retrieve the round key 5
# Using partial sums to reduce the amount of cipher texts
msgs = [[random.randint(0, 15) for _ in range(16)] for _ in range(8)]

GUESSED_KEY = [0 for _ in range(16)]

# Builds a containing the partial sums for each message
delta_sets = []

# Iterate over messages
for msg in tqdm(msgs):
    '''
    Complexity ~ 16 * 16 * 16 * 16 * (len(msgs) = 8) * 16
    '''
    # iterate over nibble positions like in the previous exercise
    # Create our partial sums
    delta_set = [[0 for _ in range(16)] for _ in range(16)]

    for nibble0 in range(16):
        for nibble1 in range(16):
            for nibble2 in range(16):
                for nibble3 in range(16):
                    m = [x for x in msg]
                    m[0] = nibble0
                    m[10] = nibble1
                    m[5] = nibble2
                    m[15] = nibble3

                    ct = midori64_c.encrypt(m, 5, keys)

                    # Revert one turn
                    ct = invShuffleCell(mixColumns(ct))

                    for i in range(16):
                        delta_set[i][ct[i]] ^= 1

    delta_sets.append(delta_set)


# Iterate over key positions
for key_pos in range(16):
    '''
    Complexity ~16 * (len(delta_sets) = 16) * 16
    '''
    # Iterate over key guesses
    for key_guess in range(16):
        for delta_set in delta_sets:
            res = 0
            for i in range(16):
                res ^= delta_set[key_pos][i] * S_BOX[i ^ key_guess]

            if res != 0:
                break

        else:
            # If it is balanced for all positions and messages it's probably right
            GUESSED_KEY[key_pos] = key_guess


100%|██████████| 8/8 [00:31<00:00,  3.88s/it]


In [11]:
assert keys[4] == mixColumns(shuffleCell(GUESSED_KEY))